In [25]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [33]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

830 documents
210 classes ['Anger Management', 'Anxiety', 'Cognitive Restructuring', 'Coping Strategies', 'Coping with Stress', 'Coping with Trauma', 'Harm Reduction', 'Identifying a Drug User', 'Legal Issues', 'Meditation Practices', 'Need for Change', 'Nutrition and Wellness', 'Overcoming Shame', 'Peer Support', 'Personal Boundaries', 'Recovery from Drug Abuse', 'Relapse Prevention', 'Resilience Building', 'Self-Care Practices', 'Social Anxiety', 'Substance Education', 'Support Systems', 'What are the types of depression?', 'Withdrawal Symptoms', 'Withdrawal Symptoms Strategies', 'about', 'academic_goal_setting', 'academic_performance', 'addiction', 'afternoon', 'anti_racism_resources', 'anxious', 'ask', 'at what age does anxiety peak?', 'body_dysmorphia_support', 'building_resilience', 'burden', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'connecting_with_peers', 'coping_strategies', 'creation', 'creative_expression', 'daily_rout

In [27]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [28]:
# shuffle our features and turn into np.array
print(len(training))
random.shuffle(training)


830


In [29]:


# training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = []
train_y = []
for i in training:
    train_x.append(i[0])
    train_y.append(i[1])
# train_x = list(training[:,0])
# train_y = list(training[:,1])
print("Training data created")


Training data created


In [30]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [31]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01,  momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy'])

In [34]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/400
166/166 [==============================] - 1s 3ms/step - loss: 1.5153 - accuracy: 0.6578 - categorical_accuracy: 0.6578 - top_k_categorical_accuracy: 0.8771
Epoch 2/400
166/166 [==============================] - 1s 5ms/step - loss: 1.3943 - accuracy: 0.6663 - categorical_accuracy: 0.6663 - top_k_categorical_accuracy: 0.8831
Epoch 3/400
166/166 [==============================] - 1s 3ms/step - loss: 1.5062 - accuracy: 0.6518 - categorical_accuracy: 0.6518 - top_k_categorical_accuracy: 0.8747
Epoch 4/400
166/166 [==============================] - 1s 4ms/step - loss: 1.6473 - accuracy: 0.6458 - categorical_accuracy: 0.6458 - top_k_categorical_accuracy: 0.8651
Epoch 5/400
166/166 [==============================] - 1s 3ms/step - loss: 1.4789 - accuracy: 0.6446 - categorical_accuracy: 0.6446 - top_k_categorical_accuracy: 0.8675
Epoch 6/400
166/166 [==============================] - 0s 2ms/step - loss: 1.5960 - accuracy: 0.6169 - categorical_accuracy: 0.6169 - top_k_categorical_acc

In [43]:
result = model.evaluate(train_x[0:20], train_y[0:20], verbose=0)

In [65]:
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'sklearn'

In [12]:
acc = hist.history['accuracy']
val_acc = hist.history['categorical_accuracy']
loss = hist.history['loss']
val_loss = hist.history['top_k_categorical_accuracy']

In [17]:
dictionary = {
    "acc": acc,
    "val_acc": val_acc,
    "loss": loss,
    "top_k_categorical_accuracy": val_loss
}

# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [15]:
import matplotlib.pyplot as plt

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

plt.show()

ModuleNotFoundError: No module named 'matplotlib'